# Building autoregression model to predict the air quality in Dar es Salaam.

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg

In [ ]:
#create client access to mongoDB server
client = MongoClient(host='localhost', port=27017)
db = client['air-quality']
dar = db['dar-es-salaam']

In [ ]:
#build wrangle function to clean dataset
def wrangle(collection):
    results = collection.find({'metadata.site': 11, 'metadata.measurement': 'P2'},
                             projection={'P2':1, 'timestamp': 1, '_id':0})
    
    df = pd.DataFrame(list(results)).set_index('timestamp')
    df.index = df.index.tz_localize('UTC').tz_convert('Africa/Dar_es_Salaam')
    
    df = df[df['P2'] <= 100]
    
    y = df['P2'].resample('1H').mean().fillna(method='ffill')
    return y

In [ ]:
#apply wrangle function on dar collection to return series
y = wrangle(dar)

#view dataset and its features
print(y.info())
print(y.head())

In [ ]:
# plot the time series data on PM2.5 Level
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(xlabel='Date', ylabel='PM2.5 Level', title='Dar es Salaam PM2.5 Levels', ax=ax);

In [ ]:
#apply 168(weeks) rolling avearage on the readings in y
fig, ax = plt.subplots(figsize=(15, 6))
y.rolling(168).mean().plot(ax=ax, ylabel='PM2.5', title='Dar es Salaam PM2.5 Levels, 7-Day Rolling Average', xlabel='Date')

In [ ]:
#create Autocorrelation Function(ACF) plot for the readings
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y, ax=ax)

In [ ]:
#create Partial Autocorrelation Function(ACF) plot for the readings
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y, ax=ax)
plt.xlabel=('Lag [hours]')
plt.ylabel=('Correlation Coefficient')
plt.title=('Dar es Salaam PM2.5 Readings, PACF')

In [ ]:
#split the readings dataset in training and test sets
cutoff_test = int(len(y) * .9)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
#establish the baseline mean absolut error for the model
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

In [ ]:
# Create range to test different lags
p_params = range(1, 31)
maes = []

# Iterate through all values of p in `p_params`
for p in p_params:
    model = AutoReg(y_train, lags=p).fit()
    y_pred = model.predict().dropna()
    mae = mean_absolute_error(y_train.iloc[p:], y_pred)
    maes.append(mae)

mae_series = pd.Series(maes, name="mae", index=p_params)
mae_series.head()

In [ ]:
# instatiate the model with the best hyperparameter
best_p = 25
best_model = AutoReg(y_train, lags=25).fit()
best_model

In [ ]:
#calculate training residuals for best_model
y_train_resid = (y_train - best_model.predict()).dropna()
y_train_resid.name = "residuals"
y_train_resid.head()

In [ ]:
# Plot histogram of residuals
y_train_resid.hist()
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Best Model, Training Residuals')

In [ ]:
#plot Autocorrelation Function for the residuals
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid, ax=ax)

In [ ]:
#perform walk-forward validation for the test data with the best_model
y_pred_wfv = pd.Series()
history = y_train.copy()

for i in range(len(y_test)):
    model = AutoReg(history, lags=best_p).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
    
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

In [ ]:
#plot the predictions against the test data
df_pred_test = pd.DataFrame({'y_test': y_test, 'y_pred_wfv': y_pred_wfv}, index=y_test.index)
fig = px.line(df_pred_test, labels={'values': 'PM2.5'})
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level")